In [682]:
import requests
from datetime import datetime
import re
import pandas as pd
from sqlalchemy import engine as sql
import json

pd.set_option('display.max_columns', 100)

In [683]:
# Функция возвращает массив всех возможных area_id
# Это необходимо для дальнейшей детализации запросов по вакансиям, т.к. API HH не возвращает более 2000 записей

def GetLeafs(element, areas_leafs):
    areas = element['areas']
    if len(areas) == 0:
        areas_leafs.append(element['id'])
    else:
        for l in range(len(areas)):    
            areas_leafs(areas[l], areas_leafs)
            
def GetAreas():       
    areas_leafs = []
    areas_Json = requests.get('https://api.hh.ru/areas').json()
    for e in range(len(areas_Json)):
        getLeafs(areas_Json[e], areas_leafs)
    return areas_leafs

In [684]:
# Функция для получения id всех вакансий, удовлетворяющих условиям. На вход принимает в виде параметра
# text - Переданное значение ищется в полях вакансии, указанных в параметре search_field.
# areas - регионы для поиска

def GetVacanciesId(text, areas):
    vacanciesId = set()

    for area in areas:
        params = {
        'text': text,
        'area': area,
        'page': 0, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
        }

        req = requests.get('https://api.hh.ru/vacancies', params).json()
        if req['found'] == 0:
            continue
        else:
            for p in range(req['pages']):
                params['page'] = p
                reqp = requests.get('https://api.hh.ru/vacancies', params).json()
                for item in reqp['items']:
                    vacanciesId.add(item['id'])
    return vacanciesId

In [685]:
# Функция возвращает датафрейм с данными по списку вакансий
def GetVacanciesData(vacancies_id, vacancy_type):

    df_index = ['id','vacancy_type', 'name', 'schedule','experience','area_name','url','employment','salary_from',\
                'salary_to','salary_currency','salary_gross','archived','created_date','published_date','emloyer_name',\
                'employer_url','employer_trusted','status','has_test','premium']
    df_vacancies = pd.DataFrame(columns=df_index)

    df_index = ['id','name']
    df_key_skills = pd.DataFrame(columns=df_index)

    for vacancy_id in vacancies_id:
        req = requests.get('https://api.hh.ru/vacancies/{}'.format(vacancy_id)).json()
        id = req['id']
        vacancy_type = vacancy_type
        name = req['name']
        try:
            schedule = req['schedule']['name']
        except:
            schedule = None
        try:
            experience = req['experience']['name']
        except:  
            experience = None
        try:
            area_name = req['area']['name']
        except:
            area_name = None
        url = req['alternate_url']
        try:    
            employment = req['employment']['name']
        except:
            employment = None
        try:    
            salary_from = req['salary']['from']
        except:
            salary_from = None
        try:    
            salary_to = req['salary']['to']
        except:
            salary_to = None
        try:    
            salary_currency = req['salary']['currency']
        except:
            salary_currency = None
        try:    
            salary_gross = req['salary']['gross']
        except:
            salary_gross = None
        archived = req['archived']
        created_date = req['created_at'][0:10]
        published_date = req['published_at'][0:10]
        try:    
            emloyer_name = req['employer']['name']
        except:
            emloyer_name = None
        try:    
            employer_url = req['employer']['alternate_url']
        except:
            employer_url = None
        try:    
            employer_trusted = req['employer']['trusted']
        except:
            employer_trusted = None
        try:    
            status = req['type']['name']
        except:
            status = None
        has_test = req['has_test']
        premium = req['premium']

        vacancy_data = [id, vacancy_type, name, schedule, experience, area_name, url, employment, salary_from,\
                    salary_to, salary_currency, salary_gross, archived, created_date, published_date, emloyer_name,\
                    employer_url, employer_trusted, status, has_test, premium]

        df_vacancies.loc[len(df_vacancies.index)] = vacancy_data
        
        key_skills = req['key_skills']
        for k in key_skills:
            key_skills_data = [id, k['name']]
            df_key_skills.loc[len(df_key_skills.index)] = key_skills_data
    
    return df_vacancies, df_key_skills  

In [686]:
text = 'NAME:"1С" AND (программист OR архитектор OR проектировщик OR разработчик)'
areas = GetAreas()
areas = areas[0:10]
vacancies_id = GetVacanciesId(text, areas)
print('Всего найдено вакансий для программиста 1С: {}'.format(len(vacancies_id)))

Всего найдено вакансий для программиста 1С: 10


In [687]:
df_vacancies_1C, df_key_skills = GetVacanciesData(vacancies_id, 'программист 1С')

In [688]:
df_vacancies_1C.head()

,id,vacancy_type,name,schedule,experience,area_name,url,employment,salary_from,salary_to,salary_currency,salary_gross,archived,created_date,published_date,emloyer_name,employer_url,employer_trusted,status,has_test,premium
0,54749017,программист 1С,Программист 1С,Удаленная работа,От 3 до 6 лет,Йошкар-Ола,https://hh.ru/vacancy/54749017,Полная занятость,180000,220000,RUR,True,False,2022-04-12,2022-04-12,КРОН,https://api.hh.ru/employers/3776815,True,Открытая,False,False
1,49982659,программист 1С,Программист 1С,Полный день,Нет опыта,Йошкар-Ола,https://hh.ru/vacancy/49982659,Полная занятость,25000,50000,RUR,True,False,2022-05-04,2022-05-04,Тиражные решения 1С-Рарус,https://api.hh.ru/employers/1156565,True,Открытая,True,False
2,54773118,программист 1С,Программист 1С,Полный день,От 1 года до 3 лет,Йошкар-Ола,https://hh.ru/vacancy/54773118,Полная занятость,None,None,None,None,False,2022-04-12,2022-04-12,Omega,https://api.hh.ru/employers/1934551,True,Открытая,False,False
3,54925895,программист 1С,Консультант технической поддержки 1С,Удаленная работа,От 1 года до 3 лет,Йошкар-Ола,https://hh.ru/vacancy/54925895,Полная занятость,42000,55000,RUR,False,False,2022-04-19,2022-04-19,Дадобоев Фаррух Шарифович,https://api.hh.ru/employers/3916109,True,Открытая,False,False
4,54950181,программист 1С,Программист 1С,Полный день,От 1 года до 3 лет,Йошкар-Ола,https://hh.ru/vacancy/54950181,Полная занятость,30000,None,RUR,False,False,2022-04-20,2022-04-20,Марикоммунэнерго,https://api.hh.ru/employers/1935236,True,Открытая,False,False


In [689]:
df_key_skills.head(3)

,id,name
0,54749017,Ответственность
1,54749017,Клиентоориентированность
2,54773118,С#


Структура ответа:

In [692]:
text = 'NAME:"1С" AND (программист OR архитектор OR проектировщик OR разработчик)'
params = {
        'text': text,
        'area': area[0:10],
        'page': 0, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
        }

req = requests.get('https://api.hh.ru/vacancies', params).json()

In [693]:
req

{'items': [{'id': '55260186',
   'premium': False,
   'name': 'Руководитель проектов 1C',
   'department': None,
   'has_test': False,
   'response_letter_required': False,
   'area': {'id': '73',
    'name': 'Петрозаводск',
    'url': 'https://api.hh.ru/areas/73'},
   'salary': {'from': 90000, 'to': 140000, 'currency': 'RUR', 'gross': False},
   'type': {'id': 'open', 'name': 'Открытая'},
   'address': None,
   'response_url': None,
   'sort_point_distance': None,
   'published_at': '2022-05-06T17:16:39+0300',
   'created_at': '2022-05-06T17:16:39+0300',
   'archived': False,
   'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=55260186',
   'insider_interview': None,
   'url': 'https://api.hh.ru/vacancies/55260186?host=hh.ru',
   'alternate_url': 'https://hh.ru/vacancy/55260186',
   'relations': [],
   'employer': {'id': '2425995',
    'name': 'Неосистемы ИТ',
    'url': 'https://api.hh.ru/employers/2425995',
    'alternate_url': 'https://hh.ru/employer/24259

In [696]:
req = requests.get('https://api.hh.ru/vacancies/48164558').json()

In [697]:
req

{'errors': [{'value': 'captcha_required',
   'captcha_url': 'https://hh.ru/account/captcha?state=pxvcxBozfu7ry7R4QCetFpeicJ2Ml%2Bur%2F701s7jX2mZxCzHziryEFFjVgnI%2BvznkA0bKItNFJ2%2Fx2EszBdDV3Vax5lfqRXRsK2RcwFMXPDf%2BAGNNFH4BJatBDCtf2QOG',
   'type': 'captcha_required'}],
 'request_id': '16518659027234f32f59cee725cae409'}

Курсы валют

In [698]:
data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
print(data['Valute']['USD'])

{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 67.3843, 'Previous': 66.2378}
